In [21]:
using SQLite

In [22]:
db = SQLite.DB("chinook.sqlite")
SQLite.tables(db)

,name
,String⍰
1,Album
2,Artist
3,Customer
4,Employee
5,Genre
6,Invoice
7,InvoiceLine
8,MediaType
9,Playlist


In [26]:
sql = "select count(*) from Customer";
df = DataFrame(SQLite.Query(db,sql))
println("Number of customers : $(df[1,1])")

Number of customers : 59


In [31]:
sql = "select a.LastName, a.FirstName,";
sql *= " count(b.InvoiceId) as Invs, sum(b.Total) as Amt";
sql *= " from Customer a";
sql *= " join Invoice b on a.CustomerId = b.CustomerId";
sql *= " group by a.LastName having Amt >= 45.00";
sql *= " order by Amt desc;";

df = DataFrame(SQLite.Query(db,sql));

In [33]:
using Printf
for i in 1:size(df)[1]
  LastName  = df[:LastName][i]
  FirstName = df[:FirstName][i]
  Amt       = df[:Amt][i]
  @printf "%10s %10s %10.2f\n" LastName FirstName Amt
 end

      Holý     Helena      49.62
Cunningham    Richard      47.62
     Rojas       Luis      46.62
    Kovács   Ladislav      45.62
  O'Reilly       Hugh      45.62


In [35]:
sql = "select a.LastName, a.FirstName, d.Name as TrackName";
sql *= " from Customer a";
sql *= " join Invoice b on a.CustomerId = b.CustomerId";
sql *= " join InvoiceLine c on b.InvoiceId = c.InvoiceId";
sql *= " join Track d on c.TrackId = d.TrackId";
sql *= " where a.LastName = 'Cunningham' and a.FirstName = 'Richard'";
sql *= " limit 5;";

df = DataFrame(SQLite.Query(db,sql));

In [37]:
for i in 1:size(df)[1]
  TrackName = df[:TrackName][i]
  @printf "%s\n" TrackName
end

Radio Free Aurope
Perfect Circle
Drowning Man
Two Hearts Beat As One
Surrender


---

In [38]:
using MySQL, DataFrames

In [39]:
# Use MAMP (on OSX) so need to specify where the mysql.sock is
#
conn = MySQL.connect("localhost", "root", "LQaLxxy6", db="Chinook", 
                     unix_socket="/Applications/MAMP/tmp/mysql/mysql.sock")

MySQL Connection
------------
Host: localhost
Port: 3306
User: root
DB:   Chinook


In [56]:
df = DataFrame(MySQL.query(conn,"show tables"));
tb = df[:,1]
println("Tables in Chinook")
for i = 1:size(tb)[1]
    print(tb[i]," ")
end


┌ Warning: `MySQL.query(conn, sql)` will return a MySQL.Query in the future; to materialize the result, use `MySQL.query(conn, sql) |> columntable` or `MySQL.query(conn, sql) |> DataFrame` instead
│   caller = ip:0x0
└ @ Core :-1


Album Artist Customer Employee Genre Invoice InvoiceLine MediaType Playlist PlaylistTrack Track 

In [60]:
DataFrame(MySQL.query(conn,"select FirstName, LastName,Address,City,State from Customer where LastName like 'C%'"))

┌ Warning: `MySQL.query(conn, sql)` will return a MySQL.Query in the future; to materialize the result, use `MySQL.query(conn, sql) |> columntable` or `MySQL.query(conn, sql) |> DataFrame` instead
│   caller = ip:0x0
└ @ Core :-1


,FirstName,LastName,Address,City,State
,String,String,String⍰,String⍰,String⍰
1,Kathy,Chase,801 W 4th Street,Reno,NV
2,Richard,Cunningham,2211 W Berry Street,Fort Worth,TX


In [67]:
sql = "select a.FirstName,a.LastName, sum(b.Total) as 'Total spent'";
sql *= " from Customer a";
sql *= " join Invoice b on a.CustomerId = b.CustomerId";
sql *= " group by a.FirstName, a.LastName"
sql *= " having a.LastName like 'C%'"
MySQL.query(conn,sql) |> DataFrame

┌ Warning: `MySQL.query(conn, sql)` will return a MySQL.Query in the future; to materialize the result, use `MySQL.query(conn, sql) |> columntable` or `MySQL.query(conn, sql) |> DataFrame` instead
│   caller = ip:0x0
└ @ Core :-1


,FirstName,LastName,Total spent
,String,String,DecFP…⍰
1,Kathy,Chase,37.62
2,Richard,Cunningham,47.62


In [20]:
MySQL.disconnect(conn)